In [1]:
import pyterrier as pt
import os
import numpy as np
import pandas as pd
import fastrank
import requests

import datetime

In [2]:
if not pt.started():
    pt.init()

PyTerrier 0.9.1 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


## Downloading the trec covid dataset

In [3]:
dataset = pt.datasets.get_dataset('irds:cord19/trec-covid')
pt_index_path = './indices/cord19'

if not os.path.exists(pt_index_path + "/data.properties"):
    indexer = pt.index.IterDictIndexer(pt_index_path, blocks=True)
    index_ref = indexer.index(dataset.get_corpus_iter(), 
                            fields=['title', 'doi', 'abstract'], 
                            meta=('docno',))
else:
    index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")
  

In [4]:
metadata = pd.read_csv('~/.ir_datasets/cord19/2020-07-16/metadata.csv')

/Users/nicolasrehbach/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
index = pt.IndexFactory.of(index_ref)
topics = dataset.get_topics('title')
qrels = dataset.get_qrels()

In [6]:
metadata.isna().sum()

cord_uid                 0
sha                 112754
source_x                 0
title                   50
doi                  55274
pmcid               105999
pubmed_id            75205
license                  0
abstract             54865
publish_time            18
authors               6477
journal              10718
mag_id              192509
who_covidence_id    142184
arxiv_id            190045
pdf_json_files      112754
pmc_json_files      129773
url                  41453
s2_id                27865
dtype: int64

In [7]:
topics

,qid,query
0,1,coronavirus origin
1,2,coronavirus response to weather changes
2,3,coronavirus immunity
3,4,how do people die from the coronavirus
4,5,animal models of covid 19
5,6,coronavirus test rapid testing
6,7,serological tests for coronavirus
7,8,coronavirus under reporting
8,9,coronavirus in canada
9,10,coronavirus social distancing impact


In [8]:
qrels

,qid,docno,label,iteration
0,1,005b2j4b,2,4.5
1,1,00fmeepz,1,4
2,1,010vptx3,2,0.5
3,1,0194oljo,1,2.5
4,1,021q9884,1,4
...,...,...,...,...
69313,50,zvop8bxh,2,5
69314,50,zwf26o63,1,5
69315,50,zwsvlnwe,0,5
69316,50,zxr01yln,1,5


In [187]:
metadata

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,clinical features of culture-proven mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636.0,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967.0,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,surfactant protein-d and pulmonary host defense,10.1186/rr19,PMC59549,11667972.0,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871.0,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888.0,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192504,z4ro6lmh,203f36475be74229101548475d68352b939f8b5b,Medline; PMC,rapid radiological improvement of covid-19 pne...,10.1007/s15010-020-01449-w,PMC7299451,32557206,no-cc,NaN,2020-06-15,"Comel, Andrea Claudio; Mosaner, Walter; Bragan...",Infection,NaN,NaN,NaN,document_parses/pdf_json/203f36475be7422910154...,document_parses/pmc_json/PMC7299451.xml.json,https://doi.org/10.1007/s15010-020-01449-w; ht...,219729576.0
192505,hi8k8wvb,9f1bc99798e8823e690697394dcb23533a45c60e,Elsevier; Medline; PMC,sars e protein in phospholipid bilayers: an an...,10.1016/j.physb.2004.11.015,PMC7127356,32288217,els-covid,Abstract We report on an anomalous X-ray refle...,2005-02-28,"Khattari, Z.; Brotons, G.; Arbely, E.; Arkin, ...",Physica B: Condensed Matter,NaN,NaN,NaN,document_parses/pdf_json/9f1bc99798e8823e69069...,document_parses/pmc_json/PMC7127356.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/32288217/;...,122247693.0
192506,ma3ndg41,ffba777376718ef2a0dd74a8eab90e2bfacd240f,Medline; PMC,italian society of interventional cardiology (...,10.1002/ccd.28888,PMC7228289,32223063,no-cc,COVID‐19 pandemic raised the issue to guarante...,2020-04-11,"Tarantini, Giuseppe; Fraccaro, Chiara; Chieffo...",Catheter Cardiovasc Interv,NaN,NaN,NaN,document_parses/pdf_json/ffba777376718ef2a0dd7...,document_parses/pmc_json/PMC7228289.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/32223063/;...,214715941.0
192507,wh10285j,d521c5a2dcbd79a5be606fcf586b1e0448344172,Medline; PMC,"nimble, together: a training program's respons...",10.1097/sla.0000000000003994,PMC7224622,32355117,cc-by-nc-nd,NaN,2020-04-29,"Bryan, Darren S.; Benjamin, Andrew J.; Schneid...",Ann Surg,NaN,NaN,NaN,d

In [61]:
len(metadata['journal'].unique())

18112

### How do pipes work

We set up a two-stage ranking pipeline with a BM25 first stage ranker whose outputs will be reranked by a ML method given the predefined featues.

In [10]:
BM25 = pt.BatchRetrieve(index, controls={"wmodel": "BM25"})
TF_IDF = pt.BatchRetrieve(index, controls = {"wmodel":"TF_IDF"})
PL2 = pt.BatchRetrieve(index, contrls = {"wmodel": "PL2"})

We make a pipe by transforming the BM25 outputs with the help of PL2 and TFIDF

In [11]:
pipe = BM25 >>(TF_IDF ** PL2)

Alternative: FeatureBatchRetrieve object

In [12]:
fbr = pt.FeaturesBatchRetrieve(index, controls = {"wmodel": "BM25"}, features=["WMODEL:TF_IDF", "WMODEL:PL2"]) 
(fbr % 5).search("coronavirus immunity")

,qid,query,docid,rank,features,docno,score
0,1,coronavirus immunity,187945,0,"[4.540221677858543, 3.477440707307063]",sp212tai,10.118259
1,1,coronavirus immunity,126990,1,"[4.377525236902022, 3.3080236155861185]",e1mw9lx1,10.001470
2,1,coronavirus immunity,179948,2,"[4.559628851404359, 3.5124279884117215]",ltmuw6f8,9.974369
3,1,coronavirus immunity,156456,3,"[4.600248221319108, 3.649953983552013]",1oruu33o,9.955978
4,1,coronavirus immunity,94922,4,"[4.490926556339275, 3.312256733881456]",5jl6ltfj,9.734640


### LTR

Split into train test using cross validation

In [13]:
train_topics, validation_topics, test_topics = np.split(topics, [int(.6*len(topics)), int(.8*len(topics))])

In [14]:
train_min = train_topics['qid'].astype(int).min()
train_max = train_topics['qid'].astype(int).max()
train_qrels = qrels[(qrels['qid'].astype(int) >= train_min) & (qrels['qid'].astype(int) <= train_max)]

val_min = validation_topics['qid'].astype(int).min()
val_max = validation_topics['qid'].astype(int).max()
validation_qrels = qrels[(qrels['qid'].astype(int) >= val_min) & (qrels['qid'].astype(int) <= val_max)]

test_min = test_topics['qid'].astype(int).min()
test_max = test_topics['qid'].astype(int).max()
test_qrels = qrels[(qrels['qid'].astype(int) >= test_min) & (qrels['qid'].astype(int) <= test_max)]

In [15]:
from sklearn.ensemble import RandomForestRegressor

In [16]:
rf = RandomForestRegressor(n_estimators=200)
rf_pipe = fbr >> pt.ltr.apply_learned_model(rf)
rf_pipe.fit(train_topics, qrels)

In [18]:
results = pt.Experiment([PL2, rf_pipe], test_topics, qrels, ["map"], names = ["PL2(Baseline)", "Random Forest"])

BR(DPH): 100%|███████████████████████████████████| 10/10 [00:00<00:00, 16.13q/s]


In [19]:
results

,name,map
0,PL2(Baseline),0.304558
1,Random Forest,0.143692


### Using Gradient Boosted Trees and LambdaMART

In [20]:
import xgboost as xgb

lmart_x = xgb.sklearn.XGBRanker(objective='rank:ndcg',
      learning_rate=0.1,
      gamma=1.0,
      min_child_weight=0.1,
      max_depth=6,
      verbose=2,
      random_state=42)

lmart_x_pipe = fbr >> pt.ltr.apply_learned_model(lmart_x, form="ltr")
lmart_x_pipe.fit(train_topics, train_qrels, validation_topics, validation_qrels)

[10:35:20] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "verbose" } are not used.



In [21]:
lmart_x_pipe = fbr >> pt.ltr.apply_learned_model(lmart_x, form="ltr")
lmart_x_pipe.fit(train_topics, train_qrels, validation_topics, validation_qrels)

train_request = fastrank.TrainRequest.coordinate_ascent()
params = train_request.params
params.init_random = True
params.normalize = True
params.seed = 1234567

ca_pipe = fbr >> pt.ltr.apply_learned_model(train_request, form="fastrank")
ca_pipe.fit(train_topics, qrels)

[10:35:59] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "verbose" } are not used.

---------------------------
Training starts...
---------------------------
[+] Random restart #1/5...
[+] Random restart #3/5...
[+] Random restart #4/5...
[+] Random restart #2/5...
[+] Random restart #5/5...
Shuffle features and optimize!
----------------------------------------
   0|Feature         |   Weight|     NDCG
----------------------------------------
Shuffle features and optimize!
----------------------------------------
   1|Feature         |   Weight|     NDCG
----------------------------------------
Shuffle features and optimize!
----------------------------------------
   2|Feature         |   Weight|     NDCG
----------------------------------------
Shuffle features and optimize!
----------------------------------------
   3|Feature         |   Weight|     NDCG
------------------------------

In [22]:
pt.Experiment(
    [PL2, lmart_x_pipe, ca_pipe],
    test_topics,
    test_qrels,
    ["map"],
    names=["PL2 Baseline", "LambdaMART (xgBoost)", 'FastRank']
)

BR(DPH): 100%|███████████████████████████████████| 10/10 [00:00<00:00, 28.69q/s]


,name,map
0,PL2 Baseline,0.304558
1,LambdaMART (xgBoost),0.167785
2,FastRank,0.236252


### Applying custom features

In [23]:
# for faster access write the author information into a dictionary
author_dict = {}
for id, authors in zip(metadata['cord_uid'], metadata['authors']):
  author_dict[id] = authors
author_dict

def authors(docno):
  
    raw_authors = author_dict[docno]
    if isinstance(raw_authors, str):
      authors = raw_authors.split(';')
      num_authors = len(authors)
      return num_authors

    return 1

def _features(row):
    f1 = authors(row["docno"])
    features = np.append(row['features'], np.array([f1]))
    return features

fbr = pt.FeaturesBatchRetrieve(index, controls = {"wmodel": "BM25"}, features=["WMODEL:TF_IDF", "WMODEL:PL2"]) 
# br = pt.BatchRetrieve(index, wmodel="BM25") >> pt.apply.doc_features(_features)

p = fbr >> pt.apply.doc_features(_features)
# p = br >> pt.apply.doc_features(_features)

p.transform("coronavirus immunity")

/Users/nicolasrehbach/opt/anaconda3/lib/python3.9/site-packages/pyterrier/ops.py:331: FutureWarning: .transform() should be passed a dataframe. Use .search() to execute a single query.
  topics = m.transform(topics)


,qid,query,docid,rank,features,docno,score
0,1,coronavirus immunity,187945,0,"[4.540221677858543, 3.477440707307063, 6.0]",sp212tai,10.118259
1,1,coronavirus immunity,126990,1,"[4.377525236902022, 3.3080236155861185, 5.0]",e1mw9lx1,10.001470
2,1,coronavirus immunity,179948,2,"[4.559628851404359, 3.5124279884117215, 5.0]",ltmuw6f8,9.974369
3,1,coronavirus immunity,156456,3,"[4.600248221319108, 3.649953983552013, 3.0]",1oruu33o,9.955978
4,1,coronavirus immunity,94922,4,"[4.490926556339275, 3.312256733881456, 3.0]",5jl6ltfj,9.734640
...,...,...,...,...,...,...,...
995,1,coronavirus immunity,107309,995,"[5.246637878274745, 4.376124849935644, 1.0]",t6gqa48n,7.153615
996,1,coronavirus immunity,118090,996,"[5.61520763991354, 5.186165183189388, 16.0]",cij94qxl,7.153615
997,1,coronavirus immunity,142138,997,"[5.379220010061081, 4.61397822144015, 20.0]",bxz9278z,7.152973
998,1,coronavirus immunity,66073,998,"[6.0197830061095505, 5.485601944147103, 26.0]",x9piyivm,7.152920


In [24]:
lmart_x_pipe = fbr >> pt.apply.doc_features(_features) >> pt.ltr.apply_learned_model(lmart_x, form="ltr")
lmart_x_pipe.fit(train_topics, train_qrels, validation_topics, validation_qrels)

[10:37:33] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "verbose" } are not used.



In [25]:
results = pt.Experiment([PL2, lmart_x_pipe], test_topics, qrels, ["map"], names=["PL2 (Baseline)", "LMART"])


BR(DPH): 100%|███████████████████████████████████| 10/10 [00:00<00:00, 29.72q/s]


In [26]:
results

,name,map
0,PL2 (Baseline),0.304558
1,LMART,0.146289


## Applying recency boost

boosts fairly well!

In [27]:
metadata['publish_year'] = pd.to_datetime(metadata['publish_time'])
metadata['publish_year'] = metadata['publish_year'].dt.strftime('%Y')
metadata['publish_year'] = metadata['publish_year'].fillna(0)
metadata['publish_year'] = metadata['publish_year'].astype(str).astype(int)

metadata['publish_year_feature'] = 0

metadata.loc[metadata['publish_year'] < 2019, 'publish_year_feature'] = 1
metadata.loc[metadata['publish_year'] == 2019, 'publish_year_feature'] = 2
metadata.loc[metadata['publish_year'] >= 2020, 'publish_year_feature'] = 2.5
metadata.loc[metadata['publish_year'] >= 2021, 'publish_year_feature'] = 3.5


In [28]:
date_dict = {}

for id, dates in zip(metadata['cord_uid'], metadata['publish_year_feature']):
    date_dict[id] = dates


In [29]:
def dates(docno):
    raw_dates = date_dict[docno]
    return raw_dates

def _features(row):
    f1 = dates(row["docno"])
    features = np.append(row['features'], np.array([f1]))
    return features

In [30]:
fbr = pt.FeaturesBatchRetrieve(index, controls = {"wmodel": "BM25"}, features=["WMODEL:TF_IDF", "WMODEL:PL2"]) 
p = fbr >> pt.apply.doc_features(_features)

In [31]:
p.transform("coronavirus immunity")

/Users/nicolasrehbach/opt/anaconda3/lib/python3.9/site-packages/pyterrier/ops.py:331: FutureWarning: .transform() should be passed a dataframe. Use .search() to execute a single query.
  topics = m.transform(topics)


,qid,query,docid,rank,features,docno,score
0,1,coronavirus immunity,187945,0,"[4.540221677858543, 3.477440707307063, 1.0]",sp212tai,10.118259
1,1,coronavirus immunity,126990,1,"[4.377525236902022, 3.3080236155861185, 2.5]",e1mw9lx1,10.001470
2,1,coronavirus immunity,179948,2,"[4.559628851404359, 3.5124279884117215, 2.5]",ltmuw6f8,9.974369
3,1,coronavirus immunity,156456,3,"[4.600248221319108, 3.649953983552013, 1.0]",1oruu33o,9.955978
4,1,coronavirus immunity,94922,4,"[4.490926556339275, 3.312256733881456, 2.5]",5jl6ltfj,9.734640
...,...,...,...,...,...,...,...
995,1,coronavirus immunity,107309,995,"[5.246637878274745, 4.376124849935644, 1.0]",t6gqa48n,7.153615
996,1,coronavirus immunity,118090,996,"[5.61520763991354, 5.186165183189388, 2.5]",cij94qxl,7.153615
997,1,coronavirus immunity,142138,997,"[5.379220010061081, 4.61397822144015, 2.5]",bxz9278z,7.152973
998,1,coronavirus immunity,66073,998,"[6.0197830061095505, 5.485601944147103, 1.0]",x9piyivm,7.152920


In [32]:
lmart_x_pipe = fbr >> pt.apply.doc_features(_features) >> pt.ltr.apply_learned_model(lmart_x, form="ltr")
lmart_x_pipe.fit(train_topics, train_qrels, validation_topics, validation_qrels)

rf = RandomForestRegressor(n_estimators=400)
rf_pipe = fbr >> pt.apply.doc_features(_features) >> pt.ltr.apply_learned_model(rf)
rf_pipe.fit(train_topics, qrels)

ca_pipe = fbr >> pt.ltr.apply_learned_model(train_request, form="fastrank")
ca_pipe.fit(train_topics, qrels)

[10:38:27] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "verbose" } are not used.

---------------------------
Training starts...
---------------------------
[+] Random restart #1/5...
[+] Random restart #3/5...
[+] Random restart #2/5...
[+] Random restart #4/5...
[+] Random restart #5/5...
Shuffle features and optimize!
----------------------------------------
   0|Feature         |   Weight|     NDCG
----------------------------------------
Shuffle features and optimize!
----------------------------------------
   2|Feature         |   Weight|     NDCG
----------------------------------------
Shuffle features and optimize!
----------------------------------------
   1|Feature         |   Weight|     NDCG
----------------------------------------
Shuffle features and optimize!
----------------------------------------
   3|Feature         |   Weight|     NDCG
------------------------------

In [33]:
results = pt.Experiment([PL2, lmart_x_pipe, rf_pipe, ca_pipe], test_topics, qrels, ["map"], names=["PL2 (Baseline)", "LMART", "RF", "Fastrank"])


BR(DPH): 100%|███████████████████████████████████| 10/10 [00:00<00:00, 30.19q/s]


In [34]:
results

,name,map
0,PL2 (Baseline),0.304558
1,LMART,0.203284
2,RF,0.171641
3,Fastrank,0.236252


## Applying Journal impact

If used: How do we cope with the missing values?

In [102]:
journal_impact = pd.read_csv('./scimagojr2020.csv', sep = ';', header = None)

journal_impact=journal_impact.T.set_index(0).T

journal_impact.isna().sum()

journal_impact = journal_impact[['Rank', 'Title', 'SJR', 'H index']]


In [103]:
merged_2 = metadata.merge(journal_impact, how = 'inner', left_on = "journal", right_on = "Title")

# Only 28279 overlapping columns. We would miss out Journal data of around 160k papers.

In [104]:
merged = metadata.merge(journal_impact, how = 'left', left_on = "journal", right_on = "Title")
merged['H index'] = [float(str(i).replace(",", "")) for i in merged['H index']]
merged['H index'] = merged['H index'].fillna(merged['H index'].mean())
merged['norm_H_index']=(merged['H index']-merged['H index'].min())/(merged['H index'].max()-merged['H index'].min())

In [105]:
# if the journal is unknown, it is placed at 4000, around place 390
# also the journals with lower SJR are going to be atleast at 4000
#merged['H index'] = merged['H index'].fillna(20)
#merged['H index'] = [float(str(i).replace(",", "")) for i in merged['normalized_SJR']]
#merged['H index'] = merged['H index'].clip(lower=5000)
#merged['H index'] = merged['normalized_SJR'].clip(upper=5001)
#merged['norm_H_index']=(merged['H index']-merged['H index'].min())/(merged['H index'].max()-merged['H index'].min())*10

In [106]:
# SJR Schwer zu benutzen

In [107]:
merged['SJR'] = [float(str(i).replace(",", "")) for i in merged['SJR']]
merged['SJR'] = merged['SJR'].fillna(merged['SJR'].mean())
merged['SJR'] = merged['SJR'].clip(upper=10000)
merged['norm_SJR']=(merged['SJR']-merged['SJR'].min())/(merged['SJR'].max()-merged['SJR'].min())


In [108]:
merged['norm_H_index'].describe()

count    192836.000000
mean          0.169446
std           0.087254
min           0.000000
25%           0.169446
50%           0.169446
75%           0.169446
max           1.000000
Name: norm_H_index, dtype: float64

In [109]:
merged['norm_SJR'].describe()

count    192836.000000
mean          0.256102
std           0.101706
min           0.000000
25%           0.261980
50%           0.261980
75%           0.261980
max           1.000000
Name: norm_SJR, dtype: float64

In [110]:
merged['journal'].nunique(), merged['title'].nunique()

(18111, 166057)

In [111]:
sjr_dict = {}
h_dict = {}

for id, sjr in zip(merged['cord_uid'], merged['norm_SJR']):
    sjr_dict[id] = sjr

for id, h_id in zip(merged['cord_uid'], merged['norm_H_index']):
    h_dict[id] = h_id


In [112]:
def dates(docno):
    raw_dates = date_dict[docno]
    return raw_dates

def sjr(docno):
    raw_sjr = sjr_dict[docno]
    return raw_sjr

def h_idx(docno):
    raw_h_idx = h_dict[docno]
    return raw_h_idx

def _features(row):
    f1 = dates(row["docno"])
    f2 = sjr(row['docno'])
    f3 = h_idx(row['docno'])
    features = np.append(row['features'], np.array([f1, f2, f3]))
#    features = np.append(row['features'], np.array([f2]))
    return features

In [99]:
fbr = pt.FeaturesBatchRetrieve(index, controls = {"wmodel": "BM25"}, features=["WMODEL:TF_IDF", "WMODEL:PL2"]) 
p = fbr >> pt.apply.doc_features(_features)

In [100]:
p.transform("coronavirus immunity")

KeyError: 'e1mw9lx1'

In [115]:
lmart_x_pipe = fbr >> pt.apply.doc_features(_features) >> pt.ltr.apply_learned_model(lmart_x, form="ltr")
lmart_x_pipe.fit(train_topics, train_qrels, validation_topics, validation_qrels)

rf = RandomForestRegressor(n_estimators=400)
rf_pipe = fbr >> pt.apply.doc_features(_features) >> pt.ltr.apply_learned_model(rf)
rf_pipe.fit(train_topics, qrels)

train_request = fastrank.TrainRequest.coordinate_ascent()
params = train_request.params
params.init_random = True
params.normalize = True
params.seed = 1234567

ca_pipe = fbr >> pt.apply.doc_features(_features) >> pt.ltr.apply_learned_model(train_request, form="fastrank")
ca_pipe.fit(train_topics, qrels)

[17:21:06] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "verbose" } are not used.

---------------------------
Training starts...
---------------------------
[+] Random restart #1/5...
[+] Random restart #3/5...
[+] Random restart #4/5...
[+] Random restart #5/5...
[+] Random restart #2/5...
Shuffle features and optimize!
----------------------------------------
   2|Feature         |   Weight|     NDCG
----------------------------------------
Shuffle features and optimize!
----------------------------------------
   3|Feature         |   Weight|     NDCG
----------------------------------------
Shuffle features and optimize!
----------------------------------------
   0|Feature         |   Weight|     NDCG
----------------------------------------
Shuffle features and optimize!
----------------------------------------
   4|Feature         |   Weight|     NDCG
------------------------------

   0|0               |   -0.295|    0.721
   2|0               |    0.000|    0.709
   1|0               |   -0.000|    0.606
   0|0               |   -0.495|    0.722
   1|0               |   -0.002|    0.610
   1|0               |   -0.005|    0.641
   2|0               |    0.000|    0.709
   1|0               |   -0.010|    0.662
   1|0               |   -0.021|    0.669
   1|0               |   -0.042|    0.670
   2|0               |   -0.000|    0.709
   1|0               |   -0.086|    0.673
   2|0               |   -0.000|    0.709
   2|0               |   -0.000|    0.709
   1|0               |   -0.172|    0.675
   1|0               |   -0.346|    0.680
   2|0               |   -0.001|    0.709
   1|0               |   -0.693|    0.684
   2|0               |   -0.008|    0.709
   1|0               |   -1.386|    0.688
   2|0               |   -0.016|    0.709
   1|0               |   -2.773|    0.692
   1|0               |   -5.547|    0.694
   2|0               |   -0.128|  

   2|2               |    0.490|    0.721
   2|2               |    0.590|    0.722
   2|2               |    0.790|    0.722
   2|2               |    1.190|    0.723
   4|1               |    0.000|    0.723
   2|2               |    1.990|    0.723
   2|2               |    3.590|    0.723
   4|1               |   -0.001|    0.723
   4|1               |   -0.003|    0.723
   2|3               |    0.001|    0.723
   4|1               |    0.000|    0.724
   2|3               |    0.001|    0.724
   2|3               |    0.002|    0.724
   2|4               |   -0.004|    0.724
   4|3               |    0.000|    0.724
   2|4               |   -0.003|    0.724
---------------------------
Shuffle features and optimize!
----------------------------------------
   4|Feature         |   Weight|     NDCG
----------------------------------------
---------------------------
Shuffle features and optimize!
----------------------------------------
   2|Feature         |   Weight|     NDCG
---

In [116]:
results = pt.Experiment([PL2, lmart_x_pipe, rf_pipe, ca_pipe], test_topics, qrels, ["map"], names=["PL2 (Baseline)", "LMART", "RF", 'Fastrank'])
#results = pt.Experiment([PL2, rf_pipe], test_topics, qrels, ["map"], names=["PL2 (Baseline)", "RF"])


BR(DPH): 100%|███████████████████████████████████| 10/10 [00:00<00:00, 31.40q/s]


In [117]:
results

,name,map
0,PL2 (Baseline),0.304558
1,LMART,0.197569
2,RF,0.171488
3,Fastrank,0.276884


### Journal Impact summary:

Little to no impact. We implemented the SRJ and H Index, it resulted in rather decreasing performance.

## Citation rank

In [322]:
fbr_df = pd.DataFrame(fbr.search("coronavirus immunity"))


In [323]:
len(fbr_df['docno'].unique())

989

In [324]:
fbr_meta = fbr_df.merge(metadata, left_on = 'docno', right_on = 'cord_uid', how = 'left')

In [325]:
fbr_meta

,qid,query,docid,rank,features,docno,score,cord_uid,sha,source_x,...,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,1,coronavirus immunity,187945,0,"[4.540221677858543, 3.477440707307063]",sp212tai,10.118259,sp212tai,281fd5c29156a80cab2f647142f635712680a58d,Medline; PMC,...,2016-03-28,"Jiang, Xinpeng; Hou, Xingyu; Tang, Lijie; Jian...",Appl Microbiol Biotechnol,NaN,NaN,NaN,document_parses/pdf_json/281fd5c29156a80cab2f6...,document_parses/pmc_json/PMC7080089.xml.json,https://doi.org/10.1007/s00253-016-7424-9; htt...,18427025.0
1,1,coronavirus immunity,126990,1,"[4.377525236902022, 3.3080236155861185]",e1mw9lx1,10.001470,e1mw9lx1,NaN,WHO,...,2020,"Li, Keying; Hao, Zhenhua; Zhao, Xiaohui; Du, J...",Scand J Immunol,NaN,#343507,NaN,NaN,NaN,NaN,218857162.0
2,1,coronavirus immunity,179948,2,"[4.559628851404359, 3.5124279884117215]",ltmuw6f8,9.974369,ltmuw6f8,3f10d19b3ba9e88daa3d92c97348e32c513df602,Medline; PMC,...,2020-05-23,"Li, Keying; Hao, Zhenhua; Zhao, Xiaohui; Du, J...",Scand J Immunol,NaN,NaN,NaN,document_parses/pdf_json/3f10d19b3ba9e88daa3d9...,NaN,https://doi.org/10.1111/sji.12895; https://www...,218857162.0
3,1,coronavirus immunity,156456,3,"[4.600248221319108, 3.649953983552013]",1oruu33o,9.955978,1oruu33o,46decd956d74e9db8e5c0f023b554015a7d8f0af; 079a...,Elsevier; Medline; PMC,...,2008-04-01,"Frieman, Matthew; Heise, Mark; Baric., Ralph",Virus Research,NaN,NaN,NaN,document_parses/pdf_json/46decd956d74e9db8e5c0...,document_parses/pmc_json/PMC2292640.xml.json,https://api.elsevier.com/content/article/pii/S...,25207793.0
4,1,coronavirus immunity,94922,4,"[4.490926556339275, 3.312256733881456]",5jl6ltfj,9.734640,5jl6ltfj,NaN,WHO,...,2020,"Hotez, Peter J; Bottazzi, Maria Elena; Corry, ...",Microbes Infect,NaN,#72285,NaN,NaN,NaN,NaN,215790279.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,1,coronavirus immunity,107309,995,"[5.246637878274745, 4.376124849935644]",t6gqa48n,7.153615,t6gqa48n,NaN,WHO,...,NaN,NaN,NaN,NaN,#11362,NaN,NaN,NaN,NaN,214601393.0
1021,1,coronavirus immunity,118090,996,"[5.61520763991354, 5.186165183189388]",cij94qxl,7.153615,cij94qxl,NaN,WHO,...,2020,"Carmo, Anália; Pereira-Vaz, João; Mota, Vanda;...",J. med. virol,NaN,#459338,NaN,NaN,NaN,NaN,219286754.0
1022,1,coronavirus immunity,142138,997,"[5.379220010061081, 4.61397822144015]",bxz9278z,7.152973,bxz9278z,1d44a32d42af0cad8b32a0535b12e47736030778,Medline; PMC; WHO,...,2020-03-15,"Graham, Jessica B; Swarts, Jessica L; Menacher...",J Infect Dis,NaN,#27190,NaN,document_parses/pdf_json/1d44a32d42af0cad8b32a...,document_parses/pmc_json/PMC7107456.xml.json,https://doi.org/10.1093/infdis/jiz531; https:/...,204757014.0
1023,1,coronavirus immunity,66073,998,"[6.0197830061095505, 5.485601944147103]",x9piyivm,7.152920,x9piyivm,NaN,Medline,...,2015,"Muthumani, Karuppiah; Falzarano, Darryl; Reusc...",Science translational medicine,NaN,NaN,NaN,NaN,NaN,https://doi.org/10.1126/scitranslmed.aac7462; ...,206688706.0


In [326]:
fbr_meta['cord_uid'].nunique()

989

In [317]:
fbr_meta = fbr_meta[['qid', 'query', 'docid', 'rank', 'features', 'docno', 'score', 'cord_uid', 'title', 'doi',
                    'abstract', 'publish_time', 'authors', 'journal']]

In [318]:
#fbr_meta = fbr_meta.drop_duplicates(subset='title', keep="first")
fbr_meta = fbr_meta.reset_index()
fbr_meta = fbr_meta.drop(columns = 'index')
#fbr_meta.update('"' + fbr_meta[['title']].astype(str) + '"')


In [319]:
#fbr_meta = len(fbr_meta['docid'])-len(fbr_meta['docid'].drop_duplicates())
fbr_meta = fbr_meta.drop_duplicates(subset='title', keep="first")
fbr_meta = fbr_meta.drop_duplicates(subset='docid', keep="first")


In [296]:
# get citation informations
def get_topic_cites(fbr_topic):
    fbr_df = pd.DataFrame(fbr.search(fbr_topic))
    # merge the fbr (pooled documents) with our metadata and remove duplicates to minimize API calls
    fbr_meta = fbr_df.merge(metadata, left_on = 'docno', right_on = 'cord_uid', how = 'left')    
    fbr_meta = fbr_meta[['qid', 'query', 'docid', 'rank', 'features', 'docno', 'score', 'cord_uid', 'title', 'doi',
                    'abstract', 'publish_time', 'authors', 'journal']]
    #fbr_meta = fbr_meta.drop_duplicates(subset='doi', keep="first")
    #fbr_meta = fbr_meta.drop_duplicates(subset='title', keep="first")
    fbr_meta = fbr_meta.reset_index()
    fbr_meta = fbr_meta.drop(columns = 'index')
    #fbr_meta.update('"' + fbr_meta[['title']].astype(str) + '"')
    fbr_meta = fbr_meta.drop_duplicates(subset='title', keep="first")

    
    # Loop through the documents and call the API to get the citation informations 
    cites_df = pd.DataFrame()
    for i in range(len(fbr_meta[:50])):
        try:
            title_information =  requests.get(
                'https://api.openalex.org/works?filter=title.search:'+fbr_meta['title'][i]
                ).json()['results'][0]
            current_title = pd.DataFrame.from_dict(title_information, orient='index')
            current_title = current_title.transpose()
            cites_df = cites_df.append(current_title)
            print(current_title['title'])
        # in case DOI and title is missing, no information is printed
        except: 
            try:
                doi_information = requests.get(
                    'https://api.openalex.org/works?filter=doi:https://doi.org/'+fbr_meta['doi'][i]
                    ).json()['results'][0]
                    #print('Retrieved:',fbr_meta['doi'][i])
                    # append the current document information to our main dataframe
                current_doi = pd.DataFrame.from_dict(doi_information, orient='index')
                current_doi = current_doi.transpose()
                cites_df = cites_df.append(current_doi)
                print(current_doi['doi'])
            except:
                print("No information")
    # finally, we are merging our OpenAlex data with our fbr and meta data by merging over the doi
    cites_df = cites_df[['doi', 'title', 'referenced_works', 'related_works', 'cited_by_count', 'counts_by_year']]
    cites_df = cites_df.reset_index()
    cites_df = cites_df.drop(columns = 'index')
    cites_df['doi'] = cites_df.doi.str.replace('https://doi.org/', '')
    #fbr_meta_cites = fbr_meta.merge(cites_df, left_on = 'title', right_on = 'title')
    return cites_df

In [297]:
cites_df = get_topic_cites('coronavirus immunity')

0    A phase trial of the oral Lactobacillus casei ...
Name: title, dtype: object
0    SARS‐CoV‐2 infection‐induced immune responses:...
Name: title, dtype: object
0    SARS‐CoV‐2 infection‐induced immune responses:...
Name: title, dtype: object
0    SARS coronavirus and innate immunity
Name: title, dtype: object
0    The potential role of Th17 immune responses in...
Name: title, dtype: object
0    Candidate Targets for Immune Responses to 2019...
Name: title, dtype: object
0    Nsp1 proteins of group I and SARS coronaviruse...
Name: title, dtype: object
No information
0    Duration of challenge immunity to coronavirus ...
Name: title, dtype: object
0    Induction of mucosal immunity in cotton rats t...
Name: title, dtype: object
No information
No information
No information
0    Lessons for COVID-19 Immunity from Other Coron...
Name: title, dtype: object
0    A Pathophysiological Perspective on COVID-19's...
Name: title, dtype: object
0    A Pathophysiological Perspective on COVID-19's

/var/folders/vc/33g3tqwn4c59qnzswhnf6ry00000gn/T/ipykernel_26825/1017400564.py:45: FutureWarning: The default value of regex will change from True to False in a future version.
  cites_df['doi'] = cites_df.doi.str.replace('https://doi.org/', '')


In [298]:
cites_df

,doi,title,referenced_works,related_works,cited_by_count,counts_by_year
0,10.1007/s00253-016-7424-9,A phase trial of the oral Lactobacillus casei ...,"[https://openalex.org/W27886363, https://opena...","[https://openalex.org/W172644496, https://open...",25,"[{'year': 2022, 'cited_by_count': 5}, {'year':..."
1,10.1111/sji.12895,SARS‐CoV‐2 infection‐induced immune responses:...,"[https://openalex.org/W1547512866, https://ope...","[https://openalex.org/W1487857142, https://ope...",27,"[{'year': 2022, 'cited_by_count': 8}, {'year':..."
2,10.1111/sji.12895,SARS‐CoV‐2 infection‐induced immune responses:...,"[https://openalex.org/W1547512866, https://ope...","[https://openalex.org/W1487857142, https://ope...",27,"[{'year': 2022, 'cited_by_count': 8}, {'year':..."
3,10.1016/j.virusres.2007.03.015,SARS coronavirus and innate immunity,"[https://openalex.org/W41083760, https://opena...","[https://openalex.org/W1556307916, https://ope...",205,"[{'year': 2022, 'cited_by_count': 17}, {'year'..."
4,10.1016/j.micinf.2020.04.005,The potential role of Th17 immune responses in...,"[https://openalex.org/W1880905016, https://ope...","[https://openalex.org/W316464541, https://open...",83,"[{'year': 2022, 'cited_by_count': 20}, {'year'..."
5,10.2139/ssrn.3541361,Candidate Targets for Immune Responses to 2019...,"[https://openalex.org/W1868870865, https://ope...","[https://openalex.org/W3011488280, https://ope...",9,"[{'year': 2022, 'cited_by_count': 1}, {'year':..."
6,10.1016/j.meegid.2010.05.014,Nsp1 proteins of group I and SARS coronaviruse...,"[https://openalex.org/W190626844, https://open...","[https://openalex.org/W1535452588, https://ope...",25,"[{'year': 2022, 'cited_by_count': 3}, {'year':..."
7,10.1007/bf01317914,Duration of challenge immunity to coronavirus ...,"[https://openalex.org/W1409028342, https://ope...","[https://openalex.org/W199036555, https://open...",17,"[{'year': 2021, 'cited_by_count': 2}, {'year':..."
8,None,Induction of mucosal immunity in cotton rats t...,"[https://openalex.org/W796562613, https://open...","[https://openalex.org/W33508033, https://opena...",9,"[{'year': 2022, 'cited_by_count': 1}, {'year':..."
9,10.1016/j.immuni.2020.07.005,Lessons for COVID-19 Immunity from Other Coron...,"[https://openalex.org/W276942807, https://open...","[https://openalex.org/W2103741738, https://ope...",238,"[{'year': 2022, 'cited_by_count': 66}, {'year'..."


In [299]:
cites_df = cites_df.drop_duplicates(subset='title', keep="first")

In [300]:
cites_df['title'] = cites_df['title'].str.lower()

/var/folders/vc/33g3tqwn4c59qnzswhnf6ry00000gn/T/ipykernel_26825/896064322.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cites_df['title'] = cites_df['title'].str.lower()


In [301]:
fbr_meta['title'] = fbr_meta['title'].str.lower()

In [302]:
meta = metadata
meta['title'] = meta['title'].str.lower()

In [303]:
meta_cites = cites_df.merge(meta, left_on = 'title', right_on = 'title', how = 'left')
meta_cites = meta_cites[['title', 'cited_by_count', 'cord_uid']]
meta_cites = meta_cites.drop_duplicates(subset='title', keep="first")

In [304]:
meta_cites.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38 entries, 0 to 45
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   title           38 non-null     object
 1   cited_by_count  38 non-null     object
 2   cord_uid        33 non-null     object
dtypes: object(3)
memory usage: 1.2+ KB


In [305]:
fbr_meta = fbr_df.merge(meta_cites, left_on = 'docno', right_on = 'cord_uid', how = 'left')

In [306]:
fbr_meta

,qid,query,docid,rank,features,docno,score,title,cited_by_count,cord_uid
0,1,coronavirus immunity,187945,0,"[4.540221677858543, 3.477440707307063]",sp212tai,10.118259,a phase trial of the oral lactobacillus casei ...,25,sp212tai
1,1,coronavirus immunity,126990,1,"[4.377525236902022, 3.3080236155861185]",e1mw9lx1,10.001470,NaN,NaN,NaN
2,1,coronavirus immunity,179948,2,"[4.559628851404359, 3.5124279884117215]",ltmuw6f8,9.974369,sars‐cov‐2 infection‐induced immune responses:...,27,ltmuw6f8
3,1,coronavirus immunity,156456,3,"[4.600248221319108, 3.649953983552013]",1oruu33o,9.955978,sars coronavirus and innate immunity,205,1oruu33o
4,1,coronavirus immunity,94922,4,"[4.490926556339275, 3.312256733881456]",5jl6ltfj,9.734640,the potential role of th17 immune responses in...,83,5jl6ltfj
...,...,...,...,...,...,...,...,...,...,...
995,1,coronavirus immunity,107309,995,"[5.246637878274745, 4.376124849935644]",t6gqa48n,7.153615,NaN,NaN,NaN
996,1,coronavirus immunity,118090,996,"[5.61520763991354, 5.186165183189388]",cij94qxl,7.153615,NaN,NaN,NaN
997,1,coronavirus immunity,142138,997,"[5.379220010061081, 4.61397822144015]",bxz9278z,7.152973,NaN,NaN,NaN
998,1,coronavirus immunity,66073,998,"[6.0197830061095505, 5.485601944147103]",x9piyivm,7.152920,NaN,NaN,NaN


In [307]:
fbr_meta = fbr_meta.drop_duplicates(subset='cord_uid', keep="first")
#fbr_meta = fbr_meta.join(meta_cites['cited_by_count'])

In [308]:
fbr_meta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34 entries, 0 to 278
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   qid             34 non-null     object 
 1   query           34 non-null     object 
 2   docid           34 non-null     int64  
 3   rank            34 non-null     int64  
 4   features        34 non-null     object 
 5   docno           34 non-null     object 
 6   score           34 non-null     float64
 7   title           33 non-null     object 
 8   cited_by_count  33 non-null     object 
 9   cord_uid        33 non-null     object 
dtypes: float64(1), int64(2), object(7)
memory usage: 2.9+ KB


In [134]:
def cites(docno):
    raw_cites = cites_dict[docno]
    return raw_cites


def _features(row):
    f4 = cites(row["docno"])
    features = np.append(row['features'], np.array([f4]))
#    features = np.append(row['features'], np.array([f2]))
    return features

In [135]:
fbr = pt.FeaturesBatchRetrieve(index, controls = {"wmodel": "BM25"}, features=["WMODEL:TF_IDF", "WMODEL:PL2"]) 
p = fbr >> pt.apply.doc_features(_features)

In [123]:
lmart_x_pipe = fbr >> pt.apply.doc_features(_features) >> pt.ltr.apply_learned_model(lmart_x, form="ltr")
lmart_x_pipe.fit(train_topics, train_qrels, validation_topics, validation_qrels)

rf = RandomForestRegressor(n_estimators=400)
rf_pipe = fbr >> pt.apply.doc_features(_features) >> pt.ltr.apply_learned_model(rf)
rf_pipe.fit(train_topics, qrels)

train_request = fastrank.TrainRequest.coordinate_ascent()
params = train_request.params
params.init_random = True
params.normalize = True
params.seed = 1234567

ca_pipe = fbr >> pt.apply.doc_features(_features) >> pt.ltr.apply_learned_model(train_request, form="fastrank")
ca_pipe.fit(train_topics, qrels)

KeyError: '75773gwg'

In [ ]:
results = pt.Experiment([PL2, lmart_x_pipe, rf_pipe, ca_pipe], test_topics, qrels, ["map"], names=["PL2 (Baseline)", "LMART", "RF", 'Fastrank'])

In [ ]:
results

In [ ]:
    
    doi_df = pd.DataFrame()
    for i in range(len(fbr_meta)):
        try:
            if(pd.isna(fbr_meta['doi'][i]) != True):
                doi_information = requests.get(
                'https://api.openalex.org/works?filter=doi:https://doi.org/'+fbr_meta['doi'][i]
                ).json()['results'][0]
                print('Retrieved:',fbr_meta['title'][i])
                # append the current document information to our main dataframe
                current_doi = pd.DataFrame.from_dict(doi_information, orient='index')
                current_doi = current_doi.transpose()
                doi_df = doi_df.append(current_doi)
            # in case the DOI is not known, we apply the title as search
            else:
                title_information = doi_information = requests.get(
                    'https://api.openalex.org/works?filter=title.search:'+fbr_meta['title'][i]
                    ).json()['results'][0]
                current_title = pd.DataFrame.from_dict(title_information, orient='index')
                current_title = current_title.transpose()
                doi_df = doi_df.append(current_title)
        # in case DOI and title is missing, no information is printed
        except: 
            print("No Information")

### Something